In [7]:
import pandas as pd 
import json 
import os 
import sys 

sys.path.append("..")
from settings import PROJECT_PATH

In [8]:
transactions = pd.read_parquet(
    os.path.join(PROJECT_PATH, "transactions_post_feature_engineering.parquet")
)

transactions_extra_info = pd.read_parquet("../transactions_extra_infos.parquet")

with open("features_used.json", "r") as f:
    feature_names = json.load(f)

with open("categorical_features_used.json", "r") as f:
    categorical_features = json.load(f)

numerical_features = [col for col in feature_names if col not in categorical_features]



FileNotFoundError: [Errno 2] No such file or directory: 'features_used.json'

In [ ]:
transactions = transactions.merge(
    transactions_extra_info[["id_transaction", "date_transaction"]],
    on="id_transaction",
    how="left",
)

transactions["anne_mois"] = transactions["date_transaction"].dt.to_period("M")

In [ ]:
transactions_pre_covid = transactions[transactions["anne_mois"] < "2020-03"]

transactions_covid = transactions[
    transactions["anne_mois"].between("2020-03", "2021-07")
]
transactions_post_covid = transactions[transactions["anne_mois"] > "2021-07"]

In [ ]:
from evidently import ColumnMapping
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset
from evidently.report import Report

# %%
# ------------- Data Preprocessing -------------
column_mapping = ColumnMapping()

column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features
column_mapping.target = CLASSIFICATION_TARGET
column_mapping.prediction = "prediction"

In [ ]:
target_drift_report = Report(metrics=[TargetDriftPreset()])

target_drift_report.run(
    reference_data=transactions_pre_covid,
    current_data=transactions_covid,
    column_mapping=column_mapping,
)

# %%
target_drift_report